Using tensorflow mirrored strategy perform distributed training. Distributed training is used to split the training workload on different GPUs on a multi GPU system.see how performance can be optimized and training times can be reduced using this approach.


In [2]:
!nvidia-smi



Thu Jan 20 19:12:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [5]:
# view available devices
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
X_train.shape

(50000, 32, 32, 3)

In [7]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [8]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

# Pre processing

In [9]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [10]:
# convert to categorical
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes= 10, dtype= 'float32'
)

y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes= 10, dtype= 'float32'
)

Model building and training

In [11]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
# convert numpy array into tf dataset
# they support batching
# efficient pipeline

train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))



In [13]:
type(train_tf_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [14]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [15]:
strategy.num_replicas_in_sync

1

In [16]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)


In [18]:
%%timeit -n1 -r1

with strategy.scope():
  gpu_model = get_model()
  gpu_model.fit(train_dataset, epochs= 50)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/50
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tens

Measure training time on a CPU``

In [19]:
%%timeit -n1 -r1 
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=50)


Epoch 1/50
200/200 [==============================] - 54s 266ms/step - loss: 1.9721 - accuracy: 0.2992
Epoch 2/50
200/200 [==============================] - 52s 262ms/step - loss: 1.8108 - accuracy: 0.3664
Epoch 3/50
200/200 [==============================] - 53s 264ms/step - loss: 1.7489 - accuracy: 0.3893
Epoch 4/50
200/200 [==============================] - 52s 262ms/step - loss: 1.7056 - accuracy: 0.4061
Epoch 5/50
162/200 [=======================>......] - ETA: 9s - loss: 1.6711 - accuracy: 0.4198 

KeyboardInterrupt: ignored